In [1]:
import os
import json
import re
from hydra.utils import instantiate
from loguru import logger
from omegaconf import DictConfig
from pymatgen.core import Structure
from tqdm import tqdm
from collections import Counter

from gensim.models.doc2vec import Doc2Vec
from mofgraph2vec.utils.saving import save_embedding
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument
from mofgraph2vec.utils.saving import save_embedding

In [2]:
with open("./rsm_bb.json") as file:
    bb_data = json.load(file)

In [3]:
len(bb_data)

3143

In [4]:
documents = []
for i, data in tqdm(enumerate(bb_data)):
    py_cif = Structure.from_file(os.path.join("../data/cifs/rsm/", bb_data[i]["name"]+".cif"))
    composition = str(py_cif.composition).split()
    opt = re.compile("([a-zA-Z]+)([0-9]+)")
    composition = [opt.match(c).groups() for c in composition]
    nodes = list(Counter(bb_data[i]["nodes"]).keys())
    linkers = list(Counter(bb_data[i]["linkers"]).keys())
    words = composition + nodes + linkers

    doc = TaggedDocument(words=words, tags=[bb_data[i]["name"]])
    documents.append(doc)

6it [00:00, 30.81it/s]/Users/xiaoqi/opt/miniconda3/envs/mofgraph2vec/lib/python3.8/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
3143it [01:19, 39.66it/s]


In [5]:
documents[2]

TaggedDocument(words=[('Cu', '6'), ('H', '36'), ('C', '84'), ('O', '24'), '[C]1O[Cu]234O[C]O[Cu]2(O1)(O[C]O3)O[C]O4', '[O]C(=O)c1cc(C([O])=O)cc(C2=Cc3cc4c(cc3=C2)C=C(c2cc(C([O])=O)cc(C([O])=O)c2)C=4)c1'], tags=['RSM1933'])

In [6]:
model = Doc2Vec(
    vector_size = 128,
    window = 100,
    min_count = 0,
    dm = 1,
    sample = 1.0,
    workers = 4,
    alpha = 1.5e-2,
    epochs = 100,
    seed = 1234
)
model.build_vocab(documents)

In [7]:
model.train(
    documents, 
    total_examples=model.corpus_count, 
    epochs=model.epochs, 
)

In [8]:
save_embedding(
    True,
    "test/tmp/", 
    model, 
    documents, 
    model.vector_size,
    None,
    None
    )